In [1]:
include("../src/Julia.jl")

pcgWorker (generic function with 1 method)

In [2]:
pwd()

"/Users/serbanstan/git/TreePCG/julia/testing"

In [3]:
graphp = "../../graphs/grid2_u_10000_1/"

"../../graphs/grid2_u_10000_1/"

In [4]:
a = readFromFile(graphp * "graph.mtx");

In [5]:
tree = []
for i in 1:3
    tr = readFromFile(graphp * "tree$(i).mtx");
    push!(tree, tr);
end

In [6]:
x = readFromFile(graphp * "x.vec");

In [7]:
la = lap(a);
x = x - mean(x)
b = la * x; b = b - mean(b);

In [8]:
numIts = 200;

@time for i in 1:3
    @time F = treeSolver(tree[i])
    @time F(b);
    
    # a run on standard Float64 data types
    @time myx64,dbg = pcgV(la, b, F, x, maxits=numIts, verbose=false);
    norm(lap(a) * myx64 - b) / norm(b)
    insert!(dbg, 1, "$(numIts) iterations")
    insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
    writeToFile(graphp * "_log_julia_tree$(i)_default.txt", dbg)
    
    # now useing higer precision data types
    for hp in [64,128,256,512,1024]
        set_bigfloat_precision(hp)
        laHP = toHighPrecision(la; precision=hp)
        treeHP = toHighPrecision(tree[i]; precision=hp)
        xHP = toHighPrecision(x; precision=hp);
        bHP = toHighPrecision(b; precision=hp);
        FHP = treeSolver(treeHP)

        @time myxHP,dbg = pcgV(laHP, bHP, FHP, xHP, maxits=numIts, verbose=true);
        insert!(dbg, 1, "$(numIts) iterations")
        insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
        writeToFile(graphp * "_log_julia_tree$(i)_$(hp).txt", dbg)
    end
end

  0.459124 seconds (365.44 k allocations: 17.986 MB, 2.48% gc time)
  0.183499 seconds (268.19 k allocations: 7.752 MB)
  2.509432 seconds (32.11 M allocations: 839.758 MB, 4.36% gc time)
Working on iteration 10 with a-norm error 3975.9428975683195
Working on iteration 20 with a-norm error 3023.9076496371085
Working on iteration 30 with a-norm error 1890.4059470485824
Working on iteration 40 with a-norm error 1208.7872650105749
Working on iteration 50 with a-norm error 693.2912459372187
Working on iteration 60 with a-norm error 397.4213917634292
Working on iteration 70 with a-norm error 221.65994352000183
Working on iteration 80 with a-norm error 134.89119133511923
Working on iteration 90 with a-norm error 81.75760717213305
Working on iteration 100 with a-norm error 46.352149834448824
Working on iteration 110 with a-norm error 25.70269371704367
Working on iteration 120 with a-norm error 14.835175421223976
Working on iteration 130 with a-norm error 8.84217398865957
Working on iteration 